# import necessary modules

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T
from torchvision.utils import save_image

import numpy as np
from matplotlib import pyplot as plt
import PIL
import model
from tqdm import tqdm

# load data and make data loader

In [2]:
cifar100_train_x_128_1 = torch.load('cifar100_train_x_128_1.pt')
cifar100_train_x_128_2 = torch.load('cifar100_train_x_128_2.pt')
cifar100_train_x_128_3 = torch.load('cifar100_train_x_128_3.pt')
cifar100_train_x_128_4 = torch.load('cifar100_train_x_128_4.pt')
cifar100_train_x_128_5 = torch.load('cifar100_train_x_128_5.pt')
cifar100_train_y_128 = torch.load('cifar100_train_y_128.pt')
cifar100_val_x_128 = torch.load('cifar100_val_x_128.pt')
cifar100_val_y_128 = torch.load('cifar100_val_y_128.pt')
cifar100_test_x_128 = torch.load('cifar100_test_x_128.pt')
cifar100_test_y_128 = torch.load('cifar100_test_y_128.pt')
cifar100_train_x_128 = torch.zeros((49000,3,128,128))
cifar100_train_x_128[0:9800,:,:,:] = cifar100_train_x_128_1
cifar100_train_x_128[9800:19600,:,:,:] = cifar100_train_x_128_2
cifar100_train_x_128[19600:29400,:,:,:] = cifar100_train_x_128_3
cifar100_train_x_128[29400:39200,:,:,:] = cifar100_train_x_128_4
cifar100_train_x_128[39200:49000,:,:,:] = cifar100_train_x_128_5

In [3]:
print(cifar100_train_x_128.size())
print(cifar100_train_y_128.size())
print(cifar100_val_x_128.size())
print(cifar100_val_y_128.size())
print(cifar100_test_x_128.size())
print(cifar100_test_y_128.size())

torch.Size([49000, 3, 128, 128])
torch.Size([49000])
torch.Size([1000, 3, 128, 128])
torch.Size([1000])
torch.Size([10000, 3, 128, 128])
torch.Size([10000])


In [4]:
# M = torch.zeros((3,128,128))
# M[:,32:96,32:96] = 1
# cifar100_train_x_128_masked = cifar100_train_x_128 * (1-M)

In [5]:
# index = 2567
# c = torch.ones((49000,3,64,64))
# save_image(cifar100_train_x_128[index].cpu(), f"./original.png", nrow=1)
# save_image(cifar100_train_x_128_masked[index].cpu(), f"./masked.png", nrow=1)
# cifar100_train_x_128_masked[:,:,32:96,32:96] = c
# save_image(cifar100_train_x_128_masked[index].cpu(), f"./masked_filled.png", nrow=1)

In [6]:
batch_size = 200
train_set = TensorDataset(cifar100_train_x_128, cifar100_train_y_128)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_set = TensorDataset(cifar100_val_x_128, cifar100_val_y_128)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)
test_set = TensorDataset(cifar100_test_x_128, cifar100_test_y_128)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

# training setup

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 100
lr = 0.0005
criterion = nn.BCELoss()

In [8]:
encoder_decoder = model.EncoderDecoder().to(device)
discriminator = model.Discriminator().to(device)
opt_enc_dec = optim.Adam(encoder_decoder.parameters(), lr=lr)
opt_disc = optim.Adam(discriminator.parameters(), lr=lr)

# training function

In [10]:
def fit(encoder_decoder, discriminator, dataloader, opt_enc_dec, opt_disc):
    encoder_decoder.train()
    discriminator.train()
    running_loss = 0.0
    M = torch.zeros((3,128,128))
    M[:,32:96,32:96] = 1
    M = M.to(device)
    for i, (data, label) in tqdm(enumerate(dataloader), total=int(len(dataloader.dataset)/dataloader.batch_size)):
        data = data.to(device) # 200, 3, 128, 128
        masked_data = data*(1-M)
        recovered_data = torch.clone(masked_data).to(device)
        gened_mask = encoder_decoder.forward(masked_data) # 200, 3, 64, 64
        recovered_data[:,:,32:96,32:96] = gened_mask # 200, 3, 128, 128
        
        ### Train Discriminator max log(D(x)) + log(1 - D(G(z)))
        disc_real = discriminator(data).reshape(-1) # N
        loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = discriminator(recovered_data).reshape(-1)
        loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        loss_disc = (loss_disc_real + loss_disc_fake) / 2
        discriminator.zero_grad()
        loss_disc.backward(retain_graph=True)
        opt_disc.step()
        
        ### train encoder_decoder
        encoder_decoder.zero_grad()
        disc_gened = discriminator(masked_data).reshape(-1)
        adv_loss = criterion(disc_gened, torch.ones_like(disc_gened))
        rec_loss = torch.norm(M * (data - recovered_data))**2
        loss = 5*adv_loss + rec_loss
        running_loss += loss.item()
        loss.backward()
        opt_enc_dec.step()
    train_loss = running_loss/len(dataloader.dataset)
    return train_loss

# validation function

In [11]:
def validate(encoder_decoder, discriminator, dataloader):
    encoder_decoder.eval()
    discriminator.eval()
    running_loss = 0.0
    M = torch.zeros((3,128,128))
    M[:,32:96,32:96] = 1
    M = M.to(device)
    with torch.no_grad():
        for i, (data, label) in tqdm(enumerate(dataloader), total=int(len(dataloader.dataset)/dataloader.batch_size)):
            data = data.to(device) # 200, 3, 128, 128
            masked_data = data*(1-M)
            gened_mask = encoder_decoder.forward(masked_data) # 200, 3, 64, 64
            masked_data[:,:,32:96,32:96] = gened_mask # 200, 3, 128, 128
            disc_gened = discriminator(masked_data).reshape(-1)
            adv_loss = criterion(disc_gened, torch.ones_like(disc_gened))
            rec_loss = torch.norm(M * (data - masked_data))**2
            loss = adv_loss + rec_loss
            running_loss += loss.item()
        
            # save the last batch input and output of every epoch
            if i == int(len(dataloader.dataset)/dataloader.batch_size) - 1:
                num_rows = 4
                both = torch.cat((data.view(batch_size, 3, 128, 128)[:4], 
                                  masked_data.view(batch_size, 3, 128, 128)[:4]))
                save_image(both.cpu(), f"./outputs/output{epoch}.png", nrow=num_rows)
    val_loss = running_loss/len(dataloader.dataset)
    return val_loss

In [12]:
train_loss = []
val_loss = []
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss = fit(encoder_decoder, discriminator, train_loader, opt_enc_dec, opt_disc)
    val_epoch_loss = validate(encoder_decoder, discriminator, val_loader)
    train_loss.append(train_epoch_loss)
    val_loss.append(val_epoch_loss)
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f"Val Loss: {val_epoch_loss:.4f}")
    torch.save(encoder_decoder.state_dict(), 'encoder_decoder.pth')
    torch.save(discriminator.state_dict(), 'discriminator.pth')

  0%|                                                                                          | 0/245 [00:00<?, ?it/s]

Epoch 1 of 100


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.56it/s]


Train Loss: 957.3588
Val Loss: 392.3694


  0%|                                                                                          | 0/245 [00:00<?, ?it/s]

...

Train Loss: 30.8683
Val Loss: 245.9942


  0%|                                                                                          | 0/245 [00:00<?, ?it/s]

Epoch 100 of 100


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.61it/s]


Train Loss: 30.5363
Val Loss: 246.1345


In [13]:
torch.save(encoder_decoder.state_dict(), 'encoder_decoder.pth')
torch.save(discriminator.state_dict(), 'discriminator.pth')

# Ablation study - training without adversial loss

# training set up

In [8]:
encoder_decoder = model.EncoderDecoder().to(device)
opt_enc_dec = optim.Adam(encoder_decoder.parameters(), lr=lr)

# training function

In [9]:
def fit_abl(encoder_decoder, dataloader, opt_enc_dec):
    encoder_decoder.train()
    running_loss = 0.0
    M = torch.zeros((3,128,128))
    M[:,32:96,32:96] = 1
    M = M.to(device)
    for i, (data, label) in tqdm(enumerate(dataloader), total=int(len(dataloader.dataset)/dataloader.batch_size)):
        data = data.to(device) # 200, 3, 128, 128
        masked_data = data*(1-M)
        recovered_data = torch.clone(masked_data).to(device)
        gened_mask = encoder_decoder.forward(masked_data) # 200, 3, 64, 64
        recovered_data[:,:,32:96,32:96] = gened_mask # 200, 3, 128, 128
        
        ### train encoder_decoder
        encoder_decoder.zero_grad()
        rec_loss = torch.norm(M * (data - recovered_data))**2
        loss = rec_loss
        running_loss += loss.item()
        loss.backward()
        opt_enc_dec.step()
    train_loss = running_loss/len(dataloader.dataset)
    return train_loss

# validation function

In [10]:
def validate_abl(encoder_decoder, dataloader):
    encoder_decoder.eval()
    running_loss = 0.0
    M = torch.zeros((3,128,128))
    M[:,32:96,32:96] = 1
    M = M.to(device)
    with torch.no_grad():
        for i, (data, label) in tqdm(enumerate(dataloader), total=int(len(dataloader.dataset)/dataloader.batch_size)):
            data = data.to(device) # 200, 3, 128, 128
            masked_data = data*(1-M)
            gened_mask = encoder_decoder.forward(masked_data) # 200, 3, 64, 64
            masked_data[:,:,32:96,32:96] = gened_mask # 200, 3, 128, 128
            rec_loss = torch.norm(M * (data - masked_data))**2
            loss = rec_loss
            running_loss += loss.item()
        
            # save the last batch input and output of every epoch
            if i == int(len(dataloader.dataset)/dataloader.batch_size) - 1:
                num_rows = 4
                both = torch.cat((data.view(batch_size, 3, 128, 128)[:4], 
                                  masked_data.view(batch_size, 3, 128, 128)[:4]))
                save_image(both.cpu(), f"./outputs/output{epoch}.png", nrow=num_rows)
    val_loss = running_loss/len(dataloader.dataset)
    return val_loss

In [11]:
train_loss = []
val_loss = []
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss = fit_abl(encoder_decoder, train_loader, opt_enc_dec)
    val_epoch_loss = validate_abl(encoder_decoder, val_loader)
    train_loss.append(train_epoch_loss)
    val_loss.append(val_epoch_loss)
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f"Val Loss: {val_epoch_loss:.4f}")
    torch.save(encoder_decoder.state_dict(), 'encoder_decoder.pth')

  0%|                                                                                          | 0/245 [00:00<?, ?it/s]

Epoch 1 of 100


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.82it/s]


Train Loss: 680.1951
Val Loss: 336.2219


  0%|▎                                                                                 | 1/245 [00:00<00:38,  6.31it/s]

...

Train Loss: 17.8050
Val Loss: 249.5494


  0%|▎                                                                                 | 1/245 [00:00<00:35,  6.90it/s]

Epoch 100 of 100


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  6.69it/s]


Train Loss: 17.8656
Val Loss: 248.0591


# testing trained model

In [18]:
trained_model = model.EncoderDecoder()
params = torch.load('encoder_decoder5xadv.pth')
trained_model.load_state_dict(params)
trained_model = trained_model.to(device=device)

In [16]:
def test_inpainting(encoder_decoder, dataloader):
    encoder_decoder.eval()
    running_loss = 0.0
    M = torch.zeros((3,128,128))
    M[:,32:96,32:96] = 1
    M = M.to(device)
    with torch.no_grad():
        for i, (data, label) in tqdm(enumerate(dataloader), total=int(len(dataloader.dataset)/dataloader.batch_size)):
            data = data.to(device) # 200, 3, 128, 128
            masked_data = data*(1-M)
            hollow_data = torch.clone(masked_data).to(device)
            gened_mask = encoder_decoder.forward(masked_data) # 200, 3, 64, 64
            masked_data[:,:,32:96,32:96] = gened_mask # 200, 3, 128, 128
        
            num_rows = 10
            for j in range(int(200 / 10)):
                both = torch.cat((data.view(200, 3, 128, 128)[j*10:j*10+10], 
                                  hollow_data.view(200, 3, 128, 128)[j*10:j*10+10],
                                  masked_data.view(200, 3, 128, 128)[j*10:j*10+10]))
                save_image(both.cpu(), f"./tests/output{i*20+j}.png", nrow=num_rows)
    return None

In [19]:
test_inpainting(trained_model, test_loader)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [03:03<00:00,  3.66s/it]
